In [1]:
import gzip
import shutil
import os
from tqdm import tqdm
from pathlib import Path

In [2]:
data_dir = f"{Path.cwd().parent.parent}/data"
data_dir

'/home/sud/dev/mRNA/data'

The dataset downloaded from GEO is a collection of compressed files. Let's uncompres them first

In [3]:
def extract_gz_files(input_folder, output_folder):
    input_path = Path(input_folder)
    output_path = Path(output_folder)
    output_path.mkdir(exist_ok=True)
    
    gz_files = list(input_path.glob("*.txt.gz"))
    print(f"Found {len(gz_files)} .txt.gz files")
     
    pbar = tqdm(gz_files, desc="Extracting files")
    for gz_file in pbar:
        output_file = output_path / gz_file.name.replace('.gz', '') 
        with gzip.open(gz_file, 'rb') as f_in:
            with open(output_file, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out) 
        pbar.set_description(f"Extracted: {gz_file.name}") 
    return output_path

In [4]:
raw_data_folder = os.path.join(data_dir, "GSE120584_RAW")
processed_data_folder = os.path.join(data_dir, "GSE120584_extracted")
extracted_path = extract_gz_files(raw_data_folder, processed_data_folder)

Found 1601 .txt.gz files


Extracted: GSM3404831_DLB_0050.txt.gz: 100%|██████████| 1601/1601 [00:02<00:00, 659.65it/s]


These expression proﬁles include 1,601 samples, which are composed of AD cases, VaD cases, DLB cases, MCI cases, and NC.

Upon visual inspection of the extracted files, they appear to be tab-separated format with some rows on top with additional information. let's check if they have same format in all files.

In [5]:
files = 0
for txt_file in extracted_path.glob("*.txt"):
    files += 1
    print(f"First 5 lines of {txt_file.name}:")
    with open(txt_file, 'r') as f:
        for _ in range(7):
            print(f.readline().strip())
    print("\n")
    if files >= 3:
        break

First 5 lines of GSM3404971_MCI_0021.txt:
Least-squares	SpotMedian Ch1=635nm		SpotMedian Ch2= nm
y=ax+b	Gain1-Gain2	Gain1-Gain3	Gain1-Gain2	Gain1-Gain3
a	0.516	0.282	-	-
b	14.492	19.759	-	-
r2	0.999810	0.999807	-	-

SpotMedian		BG_Median		Gain1		Gain2		Gain3


First 5 lines of GSM3403956_AD_0196.txt:
Least-squares	SpotMedian Ch1=635nm		SpotMedian Ch2= nm
y=ax+b	Gain1-Gain2	Gain1-Gain3	Gain1-Gain2	Gain1-Gain3
a	0.513	0.281	-	-
b	12.784	18.381	-	-
r2	0.999749	0.999629	-	-

SpotMedian		BG_Median		Gain1		Gain2		Gain3


First 5 lines of GSM3405195_NC_0213.txt:
Least-squares	SpotMedian Ch1=635nm		SpotMedian Ch2= nm
y=ax+b	Gain1-Gain2	Gain1-Gain3	Gain1-Gain2	Gain1-Gain3
a	0.518	0.284	-	-
b	10.671	15.482	-	-
r2	0.999854	0.999604	-	-

SpotMedian		BG_Median		Gain1		Gain2		Gain3




Looks similar but need to confirm by reading as tsv with pandas

Start with checking the content of the first file..


In [6]:
import pandas as pd

In [7]:
df = pd.read_csv(extracted_path / "GSM3404971_MCI_0021.txt", sep="\t", skiprows=7)

In [8]:
df.shape

(3200, 16)

In [9]:
df.head()

,Cell,Block,Column,Row,G_Name,G_ID,635nm,nm,635nm.1,nm.1,Flag_635,Flag_,Flag_635.1,Flag_ .1,Flag_635.2,Flag_ .2
0,1,1,1,1,hsa-miR-28-3p,MIMAT0004502,78.836420,-,86.401899,-,OK,-,OK,-,OK,-
1,1,1,2,1,hsa-miR-27a-5p,MIMAT0004501,76.007845,-,87.002217,-,OK,-,OK,-,OK,-
2,1,1,3,1,hsa-miR-518b,MIMAT0002844,84.801627,-,90.703367,-,OK,-,OK,-,OK,-
3,1,1,4,1,hsa-miR-520b,MIMAT0002843,68.097720,-,72.654255,-,OK,-,OK,-,OK,-
4,1,1,5,1,hsa-miR-498,MIMAT0002824,124.404836,-,124.814483,-,OK,-,OK,-,OK,-


Now lets compare with other files too..

In [10]:
columns = df.columns.tolist()
shapes = set()
pbar = tqdm(extracted_path.glob("*.txt"), desc="Comparing files")
for txt_file in pbar:
    df_ = pd.read_csv(txt_file, sep="\t", skiprows=7)
    if df_.columns.tolist() != columns:
        print(f"Columns differ in file: {txt_file.name}")
    shapes.add(df_.shape)
    pbar.set_description(f"{txt_file.name}: {df.shape}")

GSM3403778_AD_0018.txt: (3200, 16): : 1601it [00:08, 185.97it/s] 


In [11]:
shapes

{(3200, 16)}

Perfect !! all files have same column names and same shapes. This makes things easier

In [12]:
columns

['Cell',
 'Block',
 'Column',
 'Row',
 'G_Name',
 'G_ID',
 '635nm',
 ' nm',
 '635nm.1',
 ' nm.1',
 'Flag_635',
 'Flag_ ',
 'Flag_635.1',
 'Flag_ .1',
 'Flag_635.2',
 'Flag_ .2']

In [13]:
print(f"Columns: {df.columns.tolist()}")
print(f"Shape: {df.shape}")
print(f"miRNAs: {df['G_Name'].head(5).tolist()}") # Printing only first 5 miRNAs

Columns: ['Cell', 'Block', 'Column', 'Row', 'G_Name', 'G_ID', '635nm', ' nm', '635nm.1', ' nm.1', 'Flag_635', 'Flag_ ', 'Flag_635.1', 'Flag_ .1', 'Flag_635.2', 'Flag_ .2']
Shape: (3200, 16)
miRNAs: ['hsa-miR-28-3p', 'hsa-miR-27a-5p', 'hsa-miR-518b', 'hsa-miR-520b', 'hsa-miR-498']


In [14]:
df[df['Flag_635'] == 'OK'].shape # check if all OK

(3200, 16)

In [15]:
miRNA_counts = df['G_Name'].value_counts()
print("miRNA occurrence counts:")
print(miRNA_counts.head(10))

miRNA occurrence counts:
G_Name
BLANK                 320
No Probe               85
Negative Control 1     16
QC-Probe               16
S-Probe21               8
S-Probe4                8
Negative Control 3      8
Spike Control 2         8
S-Probe2                8
S-Probe1                8
Name: count, dtype: int64


In [16]:
print(f"\n635nm stats: mean={df['635nm'].mean():.2f}, std={df['635nm'].std():.2f}")
print(f"635nm.1 stats: mean={df['635nm.1'].mean():.2f}, std={df['635nm.1'].std():.2f}")



635nm stats: mean=322.85, std=1552.77
635nm.1 stats: mean=311.57, std=1505.70


In [17]:
print(f"\nFlag_635 values: {df['Flag_635'].value_counts()}")


Flag_635 values: Flag_635
OK    3200
Name: count, dtype: int64


Data looks all OK.

In [18]:
def process_single_file(file_path):
    df = pd.read_csv(file_path, sep='\t', skiprows=7)
    sample_id = file_path.stem 
    mirna_data = df[['G_Name', '635nm']].copy() 
    mirna_data = mirna_data.drop_duplicates(subset=['G_Name'], keep='first') 
    expression_series = mirna_data.set_index('G_Name')['635nm'] 
    return expression_series, sample_id

In [19]:
txt_files = list(extracted_path.glob("*.txt"))
test_file = txt_files[0]

test_series, test_sample_id = process_single_file(test_file)

print(f"Sample ID: {test_sample_id}")
print(f"Number of miRNAs: {len(test_series)}")
print(f"First 5 miRNAs and expressions:")
print(test_series.head())

Sample ID: GSM3404971_MCI_0021
Number of miRNAs: 2613
First 5 miRNAs and expressions:
G_Name
hsa-miR-28-3p      78.836420
hsa-miR-27a-5p     76.007845
hsa-miR-518b       84.801627
hsa-miR-520b       68.097720
hsa-miR-498       124.404836
Name: 635nm, dtype: float64


### Build complete expression matrix from all files

In [20]:
def build_expression_matrix(extracted_folder):
    txt_files = list(Path(extracted_folder).glob("*.txt"))
    print(f"Processing {len(txt_files)} samples...")
    all_samples = {}
    pbar = tqdm(enumerate(txt_files), total=len(txt_files), desc="Building expression matrix")
    for i, file_path in pbar:
        if i % 100 == 0:  # Progress indicator
            pbar.set_description(f"Processed {i}/{len(txt_files)} samples...")
        
        try:
            expression_series, sample_id = process_single_file(file_path)
            all_samples[sample_id] = expression_series
            
        except Exception as e:
            print(f"Error processing {file_path.name}: {e}")

    expression_matrix = pd.DataFrame(all_samples)
    
    print(f"\nFinal expression matrix shape: {expression_matrix.shape}")
    print(f"MiRNAs: {expression_matrix.shape[0]}, Samples: {expression_matrix.shape[1]}")
    
    return expression_matrix

expression_matrix = build_expression_matrix(extracted_path)

Processing 1601 samples...


Processed 1600/1601 samples...: 100%|██████████| 1601/1601 [00:08<00:00, 179.03it/s]



Final expression matrix shape: (2613, 1601)
MiRNAs: 2613, Samples: 1601


In [21]:
expression_matrix.shape

(2613, 1601)

Here we have now created a expression matrix with 2,613 mRNAs . The paper mentioned. `"a total of 2547 miRNAs were identified in the expression profiles"
` Which looks close - likely due to different filtering. But currently the goal was to understand the raw data folder. 